In [21]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from logging import getLogger

# 常に全てのカラムを表示
pd.options.display.max_columns = None

# データ取り込み

In [22]:
# 表示
imp = pd.read_csv("input/section5-ab_test_imp.csv", header="infer")
imp = pd.DataFrame(imp)

# クリック
goal = pd.read_csv("input/section5-ab_test_goal.csv", header="infer")
goal = pd.DataFrame(goal)

In [23]:
# データ中身確認
print(imp.head())
print(goal.head())

     log_date app_name   test_name test_case  user_id  transaction_id
0  2013-10-01  game-01  sales_test         B    36703           25622
1  2013-10-01  game-01  sales_test         A    44339           25623
2  2013-10-01  game-01  sales_test         B    32087           25624
3  2013-10-01  game-01  sales_test         B    10160           25625
4  2013-10-01  game-01  sales_test         B    46113           25626
     log_date app_name   test_name test_case  user_id  transaction_id
0  2013-10-01  game-01  sales_test         B    15021           25638
1  2013-10-01  game-01  sales_test         B      351           25704
2  2013-10-01  game-01  sales_test         B     8276           25739
3  2013-10-01  game-01  sales_test         B     1230           25742
4  2013-10-01  game-01  sales_test         B    17471           25743


In [24]:
# 結合
all_df = pd.merge(imp, goal, on=["transaction_id"], how="left", suffixes=('', '_g'))

In [25]:
all_df.head(500)

,log_date,app_name,test_name,test_case,user_id,transaction_id,log_date_g,app_name_g,test_name_g,test_case_g,user_id_g
0,2013-10-01,game-01,sales_test,B,36703,25622,NaN,NaN,NaN,NaN,NaN
1,2013-10-01,game-01,sales_test,A,44339,25623,NaN,NaN,NaN,NaN,NaN
2,2013-10-01,game-01,sales_test,B,32087,25624,NaN,NaN,NaN,NaN,NaN
3,2013-10-01,game-01,sales_test,B,10160,25625,NaN,NaN,NaN,NaN,NaN
4,2013-10-01,game-01,sales_test,B,46113,25626,NaN,NaN,NaN,NaN,NaN
5,2013-10-01,game-01,sales_test,A,6605,25627,NaN,NaN,NaN,NaN,NaN
6,2013-10-01,game-01,sales_test,A,346,25628,NaN,NaN,NaN,NaN,NaN
7,2013-10-01,game-01,sales_test,A,42710,25629,NaN,NaN,NaN,NaN,NaN
8,2013-10-01,game-01,sales_test,A,37194,25630,NaN,NaN,NaN,NaN,NaN
9,2013-10-01,game-01,sales_test,A,123,25631,NaN,NaN,NaN,NaN,NaN


## データ加工

In [26]:
# 日付型に変換
all_df["log_date_formatted"] = 0
all_df["log_date_formatted"] = pd.to_datetime(all_df["log_date"], format='%Y-%m-%d')

In [27]:
# 新しいカラムでフラグ化  (クリックしたか)
all_df['has_click'] = 0
all_df.loc[all_df['test_case_g'].isnull() == False, 'has_click'] = 1


## 可視化

In [28]:
# EDA
import pandas_profiling as pp
import pixiedust as px
pp.ProfileReport(all_df)


findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
/root/miniconda3/lib/python3.7/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


Number of variables,13
Number of observations,87924
Total Missing (%),27.8%
Total size in memory,9.4 MiB
Average record size in memory,112.0 B
Numeric,2
Categorical,6
Boolean,1
Date,1
Text (Unique),0
Rejected,3


In [29]:
px.display(all_df)

log_date,app_name,test_name,test_case,user_id,transaction_id,log_date_g,app_name_g,test_name_g,test_case_g,user_id_g,log_date_formatted,has_click
2013-10-26,game-01,sales_test,A,55813,63794,nan,nan,nan,nan,nan,2013-10-26 00:00:00,0
2013-10-09,game-01,sales_test,B,10515,44244,nan,nan,nan,nan,nan,2013-10-09 00:00:00,0
2013-10-29,game-01,sales_test,B,50841,86715,nan,nan,nan,nan,nan,2013-10-29 00:00:00,0
2013-10-06,game-01,sales_test,A,49756,41355,2013-10-06,game-01,sales_test,A,49756.0,2013-10-06 00:00:00,1
2013-10-07,game-01,sales_test,B,9683,54327,nan,nan,nan,nan,nan,2013-10-07 00:00:00,0
2013-10-10,game-01,sales_test,A,7495,36564,nan,nan,nan,nan,nan,2013-10-10 00:00:00,0
2013-10-20,game-01,sales_test,A,50731,58787,nan,nan,nan,nan,nan,2013-10-20 00:00:00,0
2013-10-16,game-01,sales_test,A,43779,13838,nan,nan,nan,nan,nan,2013-10-16 00:00:00,0
2013-10-09,game-01,sales_test,B,51604,44267,nan,nan,nan,nan,nan,2013-10-09 00:00:00,0
2013-10-07,game-01,sales_test,A,42836,54316,nan,nan,nan,nan,nan,2013-10-07 00:00:00,0


In [33]:
from pivottablejs import pivot_ui
pivot_ui(all_df, outfile_path="avg.html")

In [31]:
# クロス集計をする
crossed = pd.crosstab(all_df["test_case"], all_df["has_click"])
crossed

has_click,0,1
test_case,,
A,40592,3542
B,38734,5056


In [32]:
import scipy as sp
import scipy.stats

x2, p, dof, expected = sp.stats.chi2_contingency(crossed)

print("カイ二乗値は %(x2)s" %locals() )
print("確率は %(p)s" %locals() )
print("自由度は %(dof)s" %locals() )
print( expected )

if p < 0.05:
    print("有意な差があります")
else:
    print("有意な差がありません")

カイ二乗値は 308.37505289322877
確率は 4.934139633785632e-69
自由度は 1
[[39818.18029207  4315.81970793]
 [39507.81970793  4282.18029207]]
有意な差があります
